In [1]:
from IPython.display import HTML

HTML('''
<script src='//code.jquery.com/jquery-3.3.1.min.js'></script>
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div .jp-CodeCell .jp-Cell-inputWrapper').hide();
 } else {
 $('div.input').show();
 $('div .jp-CodeCell .jp-Cell-inputWrapper').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code on/off"></form>''')

In [2]:
from ipysheet import sheet, cell, row, column, cell_range
import ipywidgets as widgets
from IPython.display import display
from pandas import read_pickle

'''Depencies 
xlrd==1.2.0 to read excel'''

'Depencies \nxlrd==1.2.0 to read excel'

In [3]:
# Create laminate input tab
# Build material database
matdb = read_pickle('matdb.pkl')
options = [('-'.join((matid, desc)), str(th)) for matid, desc, th in matdb[['Mat ID', 'Description', 't']].values]

# Build Laminate input fields
nply = widgets.BoundedIntText(value=10, min=1, max=100, step=1)
sym = widgets.Button(description='Symmetric')
fill = widgets.Button(description='Fill')
columns = 3
laminate = sheet(rows=nply.value, columns=columns, column_headers='Angle ID t'.split())
theta = column(0, ['' for _ in range(nply.value)])
matl = column(1, [widgets.Dropdown(options=options) for _ in range(nply.value)])
th = column(2, [widgets.Text() for _ in range(nply.value)])
for m_cell, t_cell in zip(matl.value, th.value):
    widgets.link((m_cell, 'value'), (t_cell, 'value'))
    
# Slots
widgets.jslink((nply, 'value'), (laminate, 'rows'))
def on_nply_change(value):
    '''Update number of rows'''
    new = value['new']
    theta.row_end = new - 1
    matl.row_end = new - 1
    th.row_end = new - 1
    
    theta.value = [theta.value[i] if i < len(theta.value) else '' for i in range(new)]
    matl.value = [widgets.Dropdown(
        options=options,
        label=matl.value[i].label) if i < len(matl.value) else widgets.Dropdown(
        options=options) for i in range(new)]
    th.value = [widgets.Text(
        value=th.value[i].value) if i < len(th.value) else widgets.Text() for i in range(new)]
    for m_cell, t_cell in zip(matl.value, th.value):
        widgets.link((m_cell, 'value'), (t_cell, 'value'))

def on_sym_click(signal):
    '''Copy angle, material id values symmetrically'''
    n = nply.value//2
    if nply.value % 2:
        theta.value = theta.value[:n] + theta.value[:n+1][::-1]
        matl.value = matl.value[:n] + [widgets.Dropdown(
                options=options, 
                label=m.label) for m in matl.value[:n+1][::-1]]
    else:
        theta.value = theta.value[:n] + theta.value[:n][::-1]
        matl.value = matl.value[:n] + [widgets.Dropdown(
                options=options, 
                label=m.label) for m in matl.value[:n][::-1]]
    
    # link material, thickness
    for m_cell, t_cell in zip(matl.value[-n:], th.value[-n:]):
        widgets.link((m_cell, 'value'), (t_cell, 'value'))

def on_fill_click(signal):
    '''Fill material id from top down'''
    m = matl.value[0].label
    for mat in matl.value:
        mat.label = m
        
# Signals
nply.observe(on_nply_change, names='value')
sym.on_click(on_sym_click)
fill.on_click(on_fill_click)

# Layout
top = widgets.HBox([nply, sym, fill])
input_box = widgets.VBox([top, laminate])
# display(input_box)

In [4]:
# Create Loads Tab

nx_label = widgets.Label(value='Nx')
ny_label = widgets.Label(value='Ny')
nxy_label = widgets.Label(value='Nxy')
nx = widgets.FloatText(value=0.0)
ny = widgets.FloatText(value=0.0)
nxy = widgets.FloatText(value=0.0)
loads = widgets.HBox([widgets.VBox([nx_label, ny_label, nxy_label]), 
                     widgets.VBox([nx, ny, nxy])])
# display(loads)

In [5]:
# Put it together 

tab_contents = [input_box, loads]
tab = widgets.Tab()
tab.children = tab_contents
for i, name in enumerate(('Laminate Input', 'Loads')):
    tab.set_title(i, name)
display(tab)